In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import albumentations as albu
from torch.utils.data import TensorDataset, DataLoader,Dataset
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import torch
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import catalyst
from catalyst.contrib.data import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst import dl, utils
from catalyst.dl import SupervisedRunner
from catalyst.callbacks import DiceCallback 
from catalyst import metrics,contrib
import tqdm
# from catalyst.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback
from catalyst.callbacks import EarlyStoppingCallback
from catalyst.contrib.losses.dice import DiceLoss


E:\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] 找不到指定的模块。
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def get_img(x, folder: str='train_images'):
    """
    Return image based on image name and folder.
    """
    data_folder = f"{path}/{folder}"
    image_path = os.path.join(data_folder, x)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert an image from one color space to another,convert BGR to RGB(change the order)
    return img


def rle_decode(mask_rle: str = '', shape: tuple = (1400, 2100)):
    '''
    Decode rle encoded mask.
    
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split() # Use Spaces as delimiters
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])] # np.asarray Convert the input to an array.
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):  # an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together etc
        img[lo:hi] = 1
    return img.reshape(shape, order='F') # https://numpy.org/doc/stable/reference/generated/numpy.reshape.html demostration about reshape


def make_mask(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    """
    Create mask based on df, image name and shape.
    """
    encoded_masks = df.loc[df['im_id'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)

    for idx, label in enumerate(encoded_masks.values):
        if label is not np.nan:
            mask = rle_decode(label)
            masks[:, :, idx] = mask
            
    return masks


def to_tensor(x, **kwargs):
    """
    Convert image or mask.
    """
    # transform opencv to pytorch
    return x.transpose(2, 0, 1).astype('float32')  # >>> a = np.arange(60).reshape((4,5,3)) >>> b = a.transpose((2,0,1)) >>> b.shape (3, 4, 5)


def mask2rle(img):
    '''
    Convert mask to rle.
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()  # Flattening is a technique that is used to convert multi-dimensional arrays into a 1-D array, it is generally used in Deep Learning while feeding the 1-D array information to the classification model.
    pixels = np.concatenate([[0], pixels, [0]])  # Join a sequence of arrays along an existing axis.
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1  # Return elements chosen from x or y depending on condition
    runs[1::2] -= runs[::2] # 看不太懂
    return ' '.join(str(x) for x in runs)


def visualize(image, mask, original_image=None, original_mask=None):
    """
    Plot image and masks.
    If two pairs of images and masks are passes, show both.
    """
    fontsize = 14
    class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(1, 5, figsize=(24, 24))

        ax[0].imshow(image)
        for i in range(4):
            ax[i + 1].imshow(mask[:, :, i])
            ax[i + 1].set_title(f'Mask {class_dict[i]}', fontsize=fontsize)
    else:
        f, ax = plt.subplots(2, 5, figsize=(24, 12))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
                
        for i in range(4):
            ax[0, i + 1].imshow(original_mask[:, :, i])
            ax[0, i + 1].set_title(f'Original mask {class_dict[i]}', fontsize=fontsize)
        
        ax[1, 0].imshow(image)
        ax[1, 0].set_title('Transformed image', fontsize=fontsize)
        
        
        for i in range(4):
            ax[1, i + 1].imshow(mask[:, :, i])
            ax[1, i + 1].set_title(f'Transformed mask {class_dict[i]}', fontsize=fontsize)
            
            
def visualize_with_raw(image, mask, original_image=None, original_mask=None, raw_image=None, raw_mask=None):
    """
    Plot image and masks.
    If two pairs of images and masks are passes, show both.
    """
    fontsize = 14
    class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}

    f, ax = plt.subplots(3, 5, figsize=(24, 12))

    ax[0, 0].imshow(original_image)
    ax[0, 0].set_title('Original image', fontsize=fontsize)

    for i in range(4):
        ax[0, i + 1].imshow(original_mask[:, :, i])
        ax[0, i + 1].set_title(f'Original mask {class_dict[i]}', fontsize=fontsize)


    ax[1, 0].imshow(raw_image)
    ax[1, 0].set_title('Original image', fontsize=fontsize)

    for i in range(4):
        ax[1, i + 1].imshow(raw_mask[:, :, i])
        ax[1, i + 1].set_title(f'Raw predicted mask {class_dict[i]}', fontsize=fontsize)
        
    ax[2, 0].imshow(image)
    ax[2, 0].set_title('Transformed image', fontsize=fontsize)


    for i in range(4):
        ax[2, i + 1].imshow(mask[:, :, i])
        ax[2, i + 1].set_title(f'Predicted mask with processing {class_dict[i]}', fontsize=fontsize)
            
            
def plot_with_augmentation(image, mask, augment):
    """
    Wrapper for `visualize` function.
    """
    augmented = augment(image=image, mask=mask)
    image_flipped = augmented['image']
    mask_flipped = augmented['mask']
    visualize(image_flipped, mask_flipped, original_image=image, original_mask=mask)
    
    
sigmoid = lambda x: 1 / (1 + np.exp(-x))


def post_process(probability, threshold, min_size):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=0),
        albu.GridDistortion(p=0.5),
        albu.OpticalDistortion(p=0.5, distort_limit=2, shift_limit=0.5),
        albu.Resize(320, 640)
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(320, 640)
    ]
    return albu.Compose(test_transform)


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)


def dice(img1, img2):
    img1 = np.asarray(img1).astype(np.bool)
    img2 = np.asarray(img2).astype(np.bool)

    intersection = np.logical_and(img1, img2)  # Compute the truth value of x1 AND x2 element-wise

    return 2. * intersection.sum() / (img1.sum() + img2.sum())

In [3]:
path = '../kaggle/understanding_cloud_organization'
os.listdir(path)

['.ipynb_checkpoints',
 'sample_submission.csv',
 'test_images',
 'train.csv',
 'train_images',
 'Untitled.ipynb']

In [4]:
train = pd.read_csv(f'{path}/train.csv',nrows = 2400)
sub = pd.read_csv(f'{path}/sample_submission.csv',nrows = 2400)

In [5]:
# train.head()
sub.head()

Image_Label EncodedPixels
0    002f507.jpg_Fish           1 1
1  002f507.jpg_Flower           1 1
2  002f507.jpg_Gravel           1 1
3   002f507.jpg_Sugar           1 1
4    0035ae9.jpg_Fish           1 1

In [6]:
# Apply a function along an axis of the DataFrame
train['Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()

Fish      600
Flower    600
Gravel    600
Sugar     600
Name: Image_Label, dtype: int64

In [7]:
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()

Sugar     423
Gravel    314
Fish      304
Flower    273
Name: Image_Label, dtype: int64

In [8]:
#DataFrame.loc Access a group of rows and columns by label(s) or a boolean array.
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().value_counts()

2    256
3    178
1    132
4     34
Name: Image_Label, dtype: int64

In [9]:
train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])


sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])

In [10]:
fig = plt.figure(figsize=(25, 16))
for j, im_id in enumerate(np.random.choice(train['im_id'].unique(), 4)):
    for i, (idx, row) in enumerate(train.loc[train['im_id'] == im_id].iterrows()): # DataFrame.iterrows() Iterate over DataFrame rows as (index, Series) pairs
        print(f'j:{j}, i:{i}\n(idx,row):{(idx, row)}\nidx:{idx}\n(row):{row}')
        ax = fig.add_subplot(5, 4, j * 4 + i + 1, xticks=[], yticks=[]) # 5*4 grid, j * 4 + i + 1 th subplot
        im = Image.open(f"{path}/train_images/{row['Image_Label'].split('_')[0]}")
        plt.imshow(im)
        mask_rle = row['EncodedPixels']
        try: # label might not be there!
            mask = rle_decode(mask_rle)
        except:
            mask = np.zeros((1400, 2100))
        plt.imshow(mask, alpha=0.5, cmap='gray') # alpha表示透明度。cmap用于将标量数据转化成颜色。把标注的区域分别画在原图上
        ax.set_title(f"Image: {row['Image_Label'].split('_')[0]}. Label: {row['label']}")

j:0, i:0
(idx,row):(992, Image_Label      0bb16fb.jpg_Fish
EncodedPixels                 NaN
label                        Fish
im_id                 0bb16fb.jpg
Name: 992, dtype: object)
idx:992
(row):Image_Label      0bb16fb.jpg_Fish
EncodedPixels                 NaN
label                        Fish
im_id                 0bb16fb.jpg
Name: 992, dtype: object
j:0, i:1
(idx,row):(993, Image_Label                                     0bb16fb.jpg_Flower
EncodedPixels    1386507 406 1387907 406 1389307 395 1389705 6 ...
label                                                       Flower
im_id                                                  0bb16fb.jpg
Name: 993, dtype: object)
idx:993
(row):Image_Label                                     0bb16fb.jpg_Flower
EncodedPixels    1386507 406 1387907 406 1389307 395 1389705 6 ...
label                                                       Flower
im_id                                                  0bb16fb.jpg
Name: 993, dtype: object
j:0, i:2
(id

In [11]:
# Preparing data for modelling

In [12]:
id_mask_count = train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().\
reset_index().rename(columns={'index': 'img_id', 'Image_Label': 'count'})
print(f'id_mask_count:\n{id_mask_count}')

train_ids, valid_ids = train_test_split(id_mask_count['img_id'].values, random_state=42, stratify=id_mask_count['count'], test_size=0.1)
print(f'train_ids:{len(train_ids)}, valid_ids:{len(valid_ids)}')
test_ids = sub['Image_Label'].apply(lambda x: x.split('_')[0]).drop_duplicates().values

id_mask_count:
          img_id  count
0    155f821.jpg      4
1    015aa06.jpg      4
2    0cb6144.jpg      4
3    19807b1.jpg      4
4    0c1d048.jpg      4
..           ...    ...
595  071edac.jpg      1
596  116433b.jpg      1
597  06d6a41.jpg      1
598  114d48a.jpg      1
599  10f21b7.jpg      1

[600 rows x 2 columns]
train_ids:540, valid_ids:60


In [14]:
image_name = '0c1d048.jpg'
image = get_img(image_name)
mask = make_mask(train, image_name)

In [15]:
visualize(image, mask)

In [16]:
# try adding some augmentations
plot_with_augmentation(image, mask, albu.HorizontalFlip(p=1))
plot_with_augmentation(image, mask, albu.VerticalFlip(p=1))
plot_with_augmentation(image, mask, albu.RandomRotate90(p=1))
plot_with_augmentation(image, mask, albu.ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03))

In [17]:
# Setting up data for training in Catalyst
class CloudDataset(Dataset):
    def __init__(self, df: pd.DataFrame = None, datatype: str = 'train', img_ids: np.array = None,
                 transforms = albu.Compose([albu.HorizontalFlip(),ToTensorV2()]),
                preprocessing=None):
        self.df = df
        if datatype != 'test':
            self.data_folder = f"{path}/train_images"
        else:
            self.data_folder = f"{path}/test_images"
        self.img_ids = img_ids
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __getitem__(self, idx):
        image_name = self.img_ids[idx]
        mask = make_mask(self.df, image_name)
        image_path = os.path.join(self.data_folder, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        if self.preprocessing:
            preprocessed = self.preprocessing(image=img, mask=mask)
            img = preprocessed['image']
            mask = preprocessed['mask']
        return img, mask

    def __len__(self):
        return len(self.img_ids)

In [18]:
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'

ACTIVATION = None
model = smp.Unet(
    encoder_name=ENCODER,   #encoder_name – name of classification model (without last dense layers) used as feature extractor to build segmentation model
    encoder_weights=ENCODER_WEIGHTS,  # one of None (random initialization), imagenet (pre-training on ImageNet).
    classes=4,
    activation=ACTIVATION,
)
# Configure data preprocessing
# All models have pretrained encoders, so you have to prepare your data the same way as during weights pretraining.
# All encoders have pretrained weights. Preparing your data the same way as during weights pre-training may give your better results (higher metric score and faster convergence). But it is relevant only for 1-2-3-channels images and not necessary in case you train the whole model, not only decoder.
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [23]:
num_workers = 0
bs = 4
train_dataset = CloudDataset(df=train, datatype='train', img_ids=train_ids, transforms = get_training_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))
valid_dataset = CloudDataset(df=train, datatype='valid', img_ids=valid_ids, transforms = get_validation_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))

train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

loaders = {
    "train": train_loader,
    "valid": valid_loader
}

In [24]:
# class CustomRunner(dl.Runner):
#     def predict_batch(self, batch):
#         # model inference step
#         return self.model(batch[0].to(self.device))

#     def on_loader_start(self, runner):
#         super().on_loader_start(runner)
#         self.meters = {
#             key: metrics.AdditiveMetric(compute_on_call=False)
#             for key in ["loss", "accuracy01", "accuracy03"]
#         }

#     def handle_batch(self, batch):
#         # model train/valid step
#         # unpack the batch
#         x, y = batch
#         # run model forward pass
#         logits = self.model(x)
#         # compute the loss
#         loss = F.cross_entropy(logits, y)
#         # compute other metrics of interest
#         accuracy01, accuracy03 = metrics.accuracy(logits, y, topk=(1, 3))
#         # log metrics
#         self.batch_metrics.update(
#             {"loss": loss, "accuracy01": accuracy01, "accuracy03": accuracy03}
#         )
#         for key in ["loss", "accuracy01", "accuracy03"]:
#             self.meters[key].update(
#                 self.batch_metrics[key].item(), self.batch_size
#             )
#         # run model backward pass
#         if self.is_train_loader:
#             self.engine.backward(loss)
#             self.optimizer.step()
#             self.optimizer.zero_grad()

#     def on_loader_end(self, runner):
#         for key in ["loss", "accuracy01", "accuracy03"]:
#             self.loader_metrics[key] = self.meters[key].compute()[0]
#         super().on_loader_end(runner)


In [25]:
num_epochs = 19
logdir = "./logs/segmentation"

# model, criterion, optimizer
optimizer = torch.optim.Adam([
    {'params': model.decoder.parameters(), 'lr': 1e-2},  # lr,learning rate
    {'params': model.encoder.parameters(), 'lr': 1e-3},  
])
criterion = DiceLoss()
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)
engine = dl.Engine()

In [27]:
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    # scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        dl.DiceCallback(input_key="logits", target_key="targets"), 
        EarlyStoppingCallback(
                    loader_key="valid",
                    metric_key="loss",
                    minimize=True,
                    patience=5, 
                    min_delta=0.001)],
    logdir=logdir,
    engine=engine,
    num_epochs=num_epochs,
    verbose=True,
)

1/19 * Epoch (train):   0%|          | 0/135 [00:00<?, ?it/s]

train (1/19) dice: -1.265525221824646 | dice/_macro: -1.265525221824646 | dice/_micro: 0.09989327937364578 | loss: 0.18212022737220465 | loss/mean: 0.18212022737220465 | loss/std: 5.575068531232681 | lr: 0.01 | momentum: 0.9


1/19 * Epoch (valid):   0%|          | 0/15 [00:00<?, ?it/s]

valid (1/19) dice: 0.15749351680278778 | dice/_macro: 0.15749351680278778 | dice/_micro: 0.07983551174402237 | loss: 0.8401077826817832 | loss/mean: 0.8401077826817832 | loss/std: 0.03847754801576748 | lr: 0.01 | momentum: 0.9
* Epoch (1/19) 


2/19 * Epoch (train):   0%|          | 0/135 [00:00<?, ?it/s]

...

valid (17/19) dice: 0.6316596865653992 | dice/_macro: 0.6316596865653992 | dice/_micro: -0.22465449571609497 | loss: 2.4022467215855916 | loss/mean: 2.4022467215855916 | loss/std: 7.1215875227979835 | lr: 0.01 | momentum: 0.9
* Epoch (17/19) 


18/19 * Epoch (train):   0%|          | 0/135 [00:00<?, ?it/s]

train (18/19) dice: -0.14496666193008423 | dice/_macro: -0.14496666193008423 | dice/_micro: 0.039097439497709274 | loss: 0.9531017457997355 | loss/mean: 0.9531017457997355 | loss/std: 0.719164761734643 | lr: 0.01 | momentum: 0.9


18/19 * Epoch (valid):   0%|          | 0/15 [00:00<?, ?it/s]

valid (18/19) dice: 0.5686553716659546 | dice/_macro: 0.5686553716659546 | dice/_micro: -0.03936036303639412 | loss: 1.1776762088139852 | loss/mean: 1.1776762088139852 | loss/std: 0.6275633755838124 | lr: 0.01 | momentum: 0.9
* Epoch (18/19) 
Top models:
./logs/segmentation\checkpoints/model.0018.pth	18.0000


In [28]:
encoded_pixels = []
loaders = {"infer": valid_loader}
valid_masks = []
probabilities = np.zeros((2220, 350, 525))
for prediction in runner.predict_loader(loader=loaders["infer"]):
    for i, (batch, output) in enumerate(tqdm.tqdm(zip(valid_dataset, prediction["logits"]))):
        print(f'i:{i}\n')
        image, mask = batch
#         print(f'image:{image}\nmask:{mask}')
        for m in mask:
#             print(f'm:{m}')
            print(f'row:{len(m)},col:{len(m[0])}')
            if m.shape != (350, 525):
                m = cv2.resize(m, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            valid_masks.append(m)

        for j, probability in enumerate(output):
            print(f'output:{probability}')
            print(f'row:{len(probability)},col:{len(probability[0])}')
            probability = probability.detach().cpu().numpy()
            if probability.shape != (350, 525):
                probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            probabilities[i * 4 + j, :, :] = probability

1it [00:00,  7.24it/s]

i:0

row:320,col:640
row:320,col:640
row:320,col:640
row:320,col:640
output:tensor([[ 0.0106,  0.0737,  0.0737,  ...,  0.0737,  0.0737, -0.0034],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0925, -0.0925, -0.0519],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0927, -0.0926, -0.0519],
        ...,
        [-0.1320, -0.0925, -0.0924,  ..., -0.0933, -0.0933, -0.0522],
        [-0.1320, -0.0925, -0.0923,  ..., -0.0932, -0.0927, -0.0522],
        [-0.1688, -0.2236, -0.2236,  ..., -0.2238, -0.2237, -0.1256]],
       device='cuda:0')
row:320,col:640
output:tensor([[0.2466, 0.3775, 0.3775,  ..., 0.3775, 0.3775, 0.2777],
        [0.1995, 0.3367, 0.3367,  ..., 0.3367, 0.3367, 0.2326],
        [0.1995, 0.3367, 0.3366,  ..., 0.3365, 0.3366, 0.2326],
        ...,
        [0.1993, 0.3363, 0.3360,  ..., 0.3352, 0.3360, 0.2324],
        [0.1994, 0.3365, 0.3363,  ..., 0.3357, 0.3365, 0.2325],
        [0.0767, 0.2441, 0.2440,  ..., 0.2435, 0.2439, 0.1524]],
       device='cuda:0')
row:320,col:640
outpu

4it [00:00,  9.11it/s]

row:320,col:640
row:320,col:640
row:320,col:640
output:tensor([[ 0.0106,  0.0737,  0.0737,  ...,  0.0734,  0.0735, -0.0034],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0930, -0.0928, -0.0520],
        [-0.1320, -0.0926, -0.0924,  ..., -0.0934, -0.0929, -0.0520],
        ...,
        [-0.1320, -0.0920, -0.0918,  ..., -0.0933, -0.0930, -0.0520],
        [-0.1320, -0.0924, -0.0926,  ..., -0.0928, -0.0927, -0.0519],
        [-0.1688, -0.2234, -0.2232,  ..., -0.2236, -0.2236, -0.1255]],
       device='cuda:0')
row:320,col:640
output:tensor([[0.2466, 0.3775, 0.3775,  ..., 0.3772, 0.3774, 0.2777],
        [0.1995, 0.3367, 0.3366,  ..., 0.3362, 0.3364, 0.2325],
        [0.1995, 0.3366, 0.3363,  ..., 0.3359, 0.3364, 0.2326],
        ...,
        [0.1990, 0.3362, 0.3358,  ..., 0.3360, 0.3366, 0.2326],
        [0.1991, 0.3363, 0.3358,  ..., 0.3365, 0.3366, 0.2326],
        [0.0767, 0.2439, 0.2439,  ..., 0.2441, 0.2441, 0.1525]],
       device='cuda:0')
row:320,col:640
output:tensor([[ 0.0074,  


2it [00:00, 11.90it/s]

...

output:tensor([[-0.0399, -0.0733, -0.0733,  ..., -0.0734, -0.0733, -0.0402],
        [-0.2470, -0.2444, -0.2444,  ..., -0.2444, -0.2445, -0.1292],
        [-0.2470, -0.2444, -0.2444,  ..., -0.2448, -0.2448, -0.1293],
        ...,
        [-0.2470, -0.2444, -0.2444,  ..., -0.2462, -0.2453, -0.1293],
        [-0.2470, -0.2444, -0.2444,  ..., -0.2453, -0.2446, -0.1293],
        [-0.3472, -0.2406, -0.2406,  ..., -0.2407, -0.2407, -0.0532]],
       device='cuda:0')
row:320,col:640
i:3

row:320,col:640
row:320,col:640
row:320,col:640
row:320,col:640
output:tensor([[ 0.0106,  0.0737,  0.0737,  ...,  0.0734,  0.0736, -0.0034],
        [-0.1320, -0.0926, -0.0924,  ..., -0.0930, -0.0928, -0.0520],
        [-0.1321, -0.0925, -0.0923,  ..., -0.0934, -0.0929, -0.0520],
        ...,
        [-0.1321, -0.0925, -0.0924,  ..., -0.0929, -0.0927, -0.0519],
        [-0.1320, -0.0924, -0.0925,  ..., -0.0926, -0.0926, -0.0519],
        [-0.1688, -0.2236, -0.2235,  ..., -0.2236, -0.2236, -0.1255]],
       de


1it [00:00, 10.00it/s]

i:0

row:320,col:640
row:320,col:640
row:320,col:640
row:320,col:640
output:tensor([[ 0.0106,  0.0737,  0.0737,  ...,  0.0698,  0.0704, -0.0039],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0871, -0.0972, -0.0544],
        [-0.1320, -0.0925, -0.0925,  ...,  0.0112, -0.0690, -0.0521],
        ...,
        [-0.1320, -0.0925, -0.0925,  ...,  0.0709, -0.0221, -0.0358],
        [-0.1320, -0.0925, -0.0924,  ..., -0.0426, -0.0715, -0.0490],
        [-0.1688, -0.2236, -0.2236,  ..., -0.2161, -0.2224, -0.1255]],
       device='cuda:0')
row:320,col:640
output:tensor([[0.2466, 0.3775, 0.3775,  ..., 0.3689, 0.3771, 0.2778],
        [0.1995, 0.3367, 0.3367,  ..., 0.2767, 0.3205, 0.2323],
        [0.1995, 0.3367, 0.3367,  ..., 0.0848, 0.2724, 0.2264],
        ...,
        [0.1995, 0.3364, 0.3364,  ..., 0.2904, 0.3210, 0.2263],
        [0.1995, 0.3366, 0.3366,  ..., 0.3407, 0.3300, 0.2312],
        [0.0767, 0.2441, 0.2441,  ..., 0.2417, 0.2433, 0.1525]],
       device='cuda:0')
row:320,col:640
outpu

4it [00:00,  9.80it/s]

row:320,col:640
i:3

row:320,col:640
row:320,col:640
row:320,col:640
row:320,col:640
output:tensor([[ 0.0106,  0.0737,  0.0737,  ...,  0.0732,  0.0735, -0.0034],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0932, -0.0928, -0.0521],
        [-0.1320, -0.0925, -0.0926,  ..., -0.0936, -0.0930, -0.0520],
        ...,
        [-0.1320, -0.0925, -0.0926,  ..., -0.0932, -0.0930, -0.0520],
        [-0.1320, -0.0925, -0.0925,  ..., -0.0927, -0.0927, -0.0520],
        [-0.1688, -0.2236, -0.2236,  ..., -0.2236, -0.2236, -0.1255]],
       device='cuda:0')
row:320,col:640
output:tensor([[0.2466, 0.3775, 0.3775,  ..., 0.3770, 0.3774, 0.2777],
        [0.1995, 0.3367, 0.3367,  ..., 0.3361, 0.3364, 0.2325],
        [0.1995, 0.3367, 0.3365,  ..., 0.3356, 0.3363, 0.2326],
        ...,
        [0.1995, 0.3367, 0.3365,  ..., 0.3360, 0.3365, 0.2326],
        [0.1995, 0.3367, 0.3366,  ..., 0.3365, 0.3366, 0.2326],
        [0.0767, 0.2441, 0.2441,  ..., 0.2441, 0.2441, 0.1525]],
       device='cuda:0')
row:3

In [29]:
class_params = {}
for class_id in range(4):
    print(class_id)
    attempts = []
    for t in range(0, 100, 5):
        t /= 100
        for ms in [0, 100, 1200, 5000, 10000]:
            masks = []
            for i in range(class_id, len(probabilities), 4):
                probability = probabilities[i]
                predict, num_predict = post_process(sigmoid(probability), t, ms)
                masks.append(predict)

            d = []
            for i, j in zip(masks, valid_masks[class_id::4]):
                if (i.sum() == 0) & (j.sum() == 0):
                    d.append(1)
                else:
                    d.append(dice(i, j))

            attempts.append((t, ms, np.mean(d)))

    attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])


    attempts_df = attempts_df.sort_values('dice', ascending=False)
    print(attempts_df.head())
    best_threshold = attempts_df['threshold'].values[0]
    best_size = attempts_df['size'].values[0]
    
    class_params[class_id] = (best_threshold, best_size)

0


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


    threshold   size      dice
99       0.95  10000  0.500000
94       0.90  10000  0.500000
89       0.85  10000  0.500000
53       0.50   5000  0.488640
54       0.50  10000  0.486591
1


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


    threshold   size  dice
99       0.95  10000   0.5
78       0.75   5000   0.5
76       0.75    100   0.5
75       0.75      0   0.5
74       0.70  10000   0.5
2


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


    threshold   size      dice
64       0.60  10000  0.500113
59       0.55  10000  0.500112
69       0.65  10000  0.500106
99       0.95  10000  0.500000
84       0.80  10000  0.500000
3


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


    threshold   size  dice
99       0.95  10000  0.25
98       0.95   5000  0.25
94       0.90  10000  0.25
93       0.90   5000  0.25
89       0.85  10000  0.25


In [30]:
import gc
torch.cuda.empty_cache()
gc.collect()

114942

In [31]:
test_dataset = CloudDataset(df=sub, datatype='test', img_ids=test_ids, transforms = get_validation_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)

loaders = {"test": test_loader}

In [32]:
encoded_pixels = []
image_id = 0
for i, test_batch in enumerate(tqdm.tqdm(loaders['test'])):
    runner_out = runner.predict_batch({"features": test_batch[0].cuda()})['logits']
    for i, batch in enumerate(runner_out):
        for probability in batch:
            
            probability = probability.cpu().detach().numpy()
            if probability.shape != (350, 525):
                probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            predict, num_predict = post_process(sigmoid(probability), class_params[image_id % 4][0], class_params[image_id % 4][1])
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                r = mask2rle(predict)
                encoded_pixels.append(r)
            image_id += 1

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:22<00:00,  3.62it/s]


In [33]:
sub['EncodedPixels'] = encoded_pixels
sub.to_csv('submission.csv', columns=['Image_Label', 'EncodedPixels'], index=False)

In [34]:
sub = pd.read_csv(f'C:/Users/Yi Jiang/kaggle/submission.csv')